# Projet Walmart - Lise Gnos #
In this project we are going to predict weekly sales for some Walmart stores.
## EDA and data preprocessing ##

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [2]:
dataset = pd.read_csv("src/Walmart_Store_sales.csv")

In [3]:
print("Number of rows : {}".format(dataset.shape[0]))
print()

print("Display of dataset: ")
display(dataset.head())
print()

print("Basics statistics: ")
data_desc = dataset.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*dataset.isnull().sum()/dataset.shape[0])

Number of rows : 150

Display of dataset: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092



Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

In [4]:
# Drop lines where target values (Weekly_Sales) are missing :
to_keep = (dataset['Weekly_Sales'].isnull()==False)
dataset = dataset.loc[to_keep,:]

In [5]:
print('number of remaining rows :') #check if drop has worked
dataset.shape[0]

number of remaining rows :


136

In [6]:
#all the columns are kept because they are all useful

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         136 non-null    float64
 1   Date          118 non-null    object 
 2   Weekly_Sales  136 non-null    float64
 3   Holiday_Flag  125 non-null    float64
 4   Temperature   121 non-null    float64
 5   Fuel_Price    124 non-null    float64
 6   CPI           125 non-null    float64
 7   Unemployment  122 non-null    float64
dtypes: float64(7), object(1)
memory usage: 9.6+ KB


In [8]:
#converting date column into a date format
import datetime

In [9]:
pd.to_datetime(dataset['Date'], format="%d-%m-%Y")

0     2011-02-18
1     2011-03-25
3            NaT
4     2010-05-28
5     2010-05-28
         ...    
145   2010-06-18
146          NaT
147   2010-06-11
148   2011-08-12
149   2012-04-20
Name: Date, Length: 136, dtype: datetime64[ns]

In [10]:

def convert_date(x) :
    try :
        return datetime.datetime.strptime(str(x), "%d-%m-%Y")
    except :
        return x

In [11]:
dataset['Year'] = dataset['Date'].apply(lambda x : convert_date(x).year if (pd.isnull(x) == False) else x)
dataset['Month'] = dataset['Date'].apply(lambda x : convert_date(x).month if (pd.isnull(x) == False) else x)
dataset['Day'] = dataset['Date'].apply(lambda x : convert_date(x).day if (pd.isnull(x) == False) else x)
dataset['Weekday'] = dataset['Date'].apply(lambda x : convert_date(x).weekday() if (pd.isnull(x) == False) else x)
dataset.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Weekday
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0
5,4.0,28-05-2010,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0,4.0


In [12]:
dataset = dataset.drop('Date', axis = 1)
dataset.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Weekday
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
3,11.0,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0
5,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,2010.0,5.0,28.0,4.0


In [13]:
# Visualize pairwise dependencies
fig = px.scatter_matrix(dataset)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height=1200, width = 1200)
fig.show()

/home/lise/.local/lib/python3.10/site-packages/plotly/express/_core.py:279: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  dims = [


From this bivariate analysis, we can see that :
* the Weekday column has only one value : 4
* there is a correlation between the temperature and the month of the year, which is logical
* there is some outliers especially with unemployment

In [14]:
#Drop lines containing invalid values or outliers

col_outliers = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']

for item in col_outliers :
    to_keep = (dataset[item] <= dataset[item].mean() + 3*dataset[item].std()) \
                & (dataset[item] >= dataset[item].mean() - 3*dataset[item].std())
    dataset = dataset.loc[to_keep,:]

dataset.shape

(90, 11)

In [15]:
dataset.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,Weekday
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0
6,15.0,695396.19,0.0,69.80,4.069,134.855161,7.658,2011.0,6.0,3.0,4.0
7,20.0,2203523.20,0.0,39.93,3.617,213.023622,6.961,2012.0,2.0,3.0,4.0


In [16]:
#it seems that the Weekday column has only one value : 4
dataset['Weekday'].unique()

array([ 4., nan])

The weekday column brings no valuable information, I choose to drop it.
(it probably means that the information brought in the dataset are always recovered on Fridays, which is logical because we figure out the weekly sales)

In [17]:
dataset = dataset.drop('Weekday', axis = 1)
dataset.shape

(90, 10)

In [18]:
dataset['Holiday_Flag'].value_counts()

0.0    74
1.0     6
Name: Holiday_Flag, dtype: int64

## Baseline model (linear regression) ##

In [19]:
# Separate target variable Y from features X
target_name = 'Weekly_Sales'

Y = dataset.loc[:,target_name]
X = dataset.drop(target_name, axis = 1)

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [21]:
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [22]:
categorical_features = ['Store', 'Holiday_Flag']
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first'))
    ])

In [23]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [24]:
X_train = preprocessor.fit_transform(X_train)

X_test = preprocessor.transform(X_test)

In [25]:
model = LinearRegression()
model.fit(X_train, Y_train)

LinearRegression()

In [26]:
Y_train_pred = model.predict(X_train)

In [27]:
Y_test_pred = model.predict(X_test)

In [28]:
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

R2 score on training set :  0.9868321417045137
R2 score on test set :  0.93522163140001


The linear regression model has already a very good score, but it seems to have some overfitting, to check that I choose to do a cross validation.

In [29]:
# Perform 5-fold cross-validation to evaluate the generalized R2 score obtained with a LinearRegression model
print("5-fold cross-validation...")
regressor = LinearRegression()
scores = cross_val_score(regressor, X_train, Y_train, cv=5)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

5-fold cross-validation...
The cross-validated R2-score is :  0.9541101472556669
The standard deviation is :  0.030783765824680693


The value of the standard deviation confirms that the model overfits : 0.987 - 0.031 = 0.956 > 0.935

In [30]:
model.coef_

array([  -11462.70403287,   -57984.82941481,   717469.90539809,
          32478.49680851,    -6895.02248795,    17243.19450848,
         -49592.65139628,   271338.35587122, -1250987.49342839,
        2204172.84483816, -1227432.8857967 ,   100272.53734887,
        -619830.01541625,  -656196.70312482, -1102517.25379658,
        1798457.27505629,   219379.59544222,  2066813.65596098,
        1017347.44167178,   589257.73223952,  -577586.04438913,
         856056.66014242,   987464.42847968,  1328210.79928435,
         592403.25635525,   -53530.32293641])

In [31]:
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = pipeline.named_steps['encoder'].get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
print("Names of columns corresponding to each coefficient: ", column_names)

Names of columns corresponding to each coefficient:  ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day', 'x0_2.0', 'x0_3.0', 'x0_4.0', 'x0_5.0', 'x0_6.0', 'x0_7.0', 'x0_8.0', 'x0_9.0', 'x0_10.0', 'x0_11.0', 'x0_13.0', 'x0_14.0', 'x0_15.0', 'x0_16.0', 'x0_17.0', 'x0_18.0', 'x0_19.0', 'x0_20.0', 'x1_1.0']


In [32]:
coefs = pd.DataFrame(index = column_names, data = model.coef_.transpose(), columns=["coefficients"])
coefs

,coefficients
Temperature,-1.146270e+04
Fuel_Price,-5.798483e+04
CPI,7.174699e+05
Unemployment,3.247850e+04
Year,-6.895022e+03
Month,1.724319e+04
Day,-4.959265e+04
x0_2.0,2.713384e+05
x0_3.0,-1.250987e+06
x0_4.0,2.204173e+06


In [33]:
feature_importance = abs(coefs).sort_values(by = 'coefficients')
feature_importance

,coefficients
Year,6.895022e+03
Temperature,1.146270e+04
Month,1.724319e+04
Unemployment,3.247850e+04
Day,4.959265e+04
x1_1.0,5.353032e+04
Fuel_Price,5.798483e+04
x0_6.0,1.002725e+05
x0_11.0,2.193796e+05
x0_2.0,2.713384e+05


In [34]:
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120}, # to avoid cropping of column names
                  height=600
                 )
fig.show()

The most important coefficients are related to which store does the sales, followed by the fuel price. \
The overfitting is not surprising since there are 26 parameters to find by the model (after one-hot-encoding the stores) and there are only 90 observations remaining after preprocessing.
## Fight overfitting ##
To deal with overfitting, we are going to try the Ridge model by performing a grid search.

In [35]:
# Perform grid search
print("Grid search...")
regressor = Ridge()
# Grid of values to be tested
params = {
    'alpha': [0.0, 0.1, 0.5, 1.0] #essayer des valeurs plus petites
}
gridsearch = GridSearchCV(regressor, param_grid = params, cv = 5) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.0}
Best R2 score :  0.9541101472556708


The Ridge model does not perform better than a classic linear regression (0.954 in both cases, and best alpha = 0). Let's try to fine tune the alpha parameter.

In [36]:
# Perform grid search
print("Grid search...")
regressor = Ridge()
# Grid of values to be tested
params = {
    'alpha': np.arange(0.0, 0.001, 0.0001) #essayer des valeurs plus petites
}
gridsearch = GridSearchCV(regressor, param_grid = params, cv = 5) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.0004}
Best R2 score :  0.954284989803685


In [37]:
Y_train_pred = gridsearch.predict(X_train)
Y_test_pred = gridsearch.predict(X_test)

print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

R2 score on training set :  0.9868053871595848
R2 score on test set :  0.9362616305225582


I do a cross-validation to check if there is overfitting or not :

In [38]:
# Perform 5-fold cross-validation to evaluate the generalized R2 score obtained with a LinearRegression model
print("5-fold cross-validation...")
regressor = Ridge(alpha = 0.0004)
scores = cross_val_score(regressor, X_train, Y_train, cv=5)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

5-fold cross-validation...
The cross-validated R2-score is :  0.954284989803685
The standard deviation is :  0.03157665033866064


The Ridge model does not enable to fight overfitting : 0.987 - 0.032 = 0.955 > 0.936.  
The model does not have a better performance than the linear regression, and there is still overfitting.  
Let's try with the Lasso model :

In [39]:
# Perform grid search
print("Grid search...")
regressor = Lasso(max_iter=100000)
# Grid of values to be tested
params = {
    'alpha': np.arange(380, 400, 1)
}
gridsearch = GridSearchCV(regressor, param_grid = params, cv = 5, verbose = 1) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best R2 score : ", gridsearch.best_score_)

Grid search...
Fitting 5 folds for each of 20 candidates, totalling 100 fits
...Done.
Best hyperparameters :  {'alpha': 394}
Best R2 score :  0.9530711689460014


In [40]:
print("LASSO / R2 score on training set : ", gridsearch.score(X_train, Y_train))
print("LASSO / R2 score on test set : ", gridsearch.score(X_test, Y_test))

LASSO / R2 score on training set :  0.9855951378641284
LASSO / R2 score on test set :  0.9390080048540245


In [41]:
# the model seems to have some overfitting, to check that I choose to do a cross validation
# Perform 5-fold cross-validation to evaluate the generalized R2 score obtained with a LinearRegression model
print("5-fold cross-validation...")
regressor = Lasso(alpha = 394, max_iter = 100000)
scores = cross_val_score(regressor, X_train, Y_train, cv=5)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

5-fold cross-validation...
The cross-validated R2-score is :  0.9530711689460014
The standard deviation is :  0.034333989343538956


In [42]:
coefsL = pd.DataFrame(index = column_names, data = gridsearch.best_estimator_.coef_.transpose(), columns=["coefficients"])
coefsL

,coefficients
Temperature,-8.338319e+03
Fuel_Price,-4.806229e+04
CPI,7.677576e+04
Unemployment,3.459514e+04
Year,3.645825e+04
Month,2.491334e+04
Day,-4.681465e+04
x0_2.0,2.508037e+05
x0_3.0,-1.160408e+06
x0_4.0,7.719247e+05


In [43]:
feature_importance = abs(coefsL).sort_values(by = 'coefficients')
feature_importance

,coefficients
x0_19.0,0.000000e+00
Temperature,8.338319e+03
Month,2.491334e+04
Unemployment,3.459514e+04
Year,3.645825e+04
x1_1.0,3.785659e+04
Day,4.681465e+04
Fuel_Price,4.806229e+04
CPI,7.677576e+04
x0_6.0,1.003080e+05


In [44]:
import plotly.express as px
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120}, # to avoid cropping of column names
                  height=600
                 )
fig.show()

The Lasso model did not seem to select some features compared to the linear regression.  
Normally, it is supposed to identify the features that are not relevant for the model. We can suppose that all the explanatory features of our model are useful.  
Besides, the train and test scores are already very high, which makes it very difficult to fight this small overfitting.